In [7]:
import os
import sys
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
from scipy.stats import ttest_1samp


In [8]:
warnings.filterwarnings('ignore')
path = os.getenv("ROOT_PATH")
sys.path.append(path)
print(path)


/Users/monic/Desktop/Master_Thesis/empirical


## Statistical Significance - Table 3a - Mid

The first step is creating a table with the daily ARs calculated using CAPM for each company

In [9]:
window_size = 250
risk_free_rate_df= pd.read_excel(f"{path}/raw_data/main/risk_free.xlsx")
risk_free_rate_df['Swedish Treasury Bills (SE TB 1 Month)'].fillna(method='ffill', inplace=True)
risk_free_rate_df['Swedish Treasury Bills (SE TB 1 Month)']= (1 + risk_free_rate_df['Swedish Treasury Bills (SE TB 1 Month)']) ** (1/250) - 1
df_price_2013 = pd.read_excel(f"{path}/raw_data/main/price_2013.xlsx")

ARs_df = pd.merge(df_price_2013,risk_free_rate_df,left_on='Date',right_on='Date',how='left')
ARs_df['market_returns'] = df_price_2013['^OMXSPI'].pct_change()
ARs_df.drop(columns='^OMXSPI',inplace=True)


In [10]:
ARs_df.head()


,Date,8TRA.ST,AAK.ST,ABB.ST,ABLI.ST,ACAD.ST,ACE.ST,ACRI-A.ST,ACRI-B.ST,ACTI.ST,...,WBGR-B.ST,WIHL.ST,WISE.ST,WTW-A.ST,XANO-B.ST,XBRANE.ST,XSPRAY.ST,XVIVO.ST,Swedish Treasury Bills (SE TB 1 Month),market_returns
0,2013-03-06,NaN,42.981308,134.692978,17.437082,NaN,NaN,NaN,NaN,30.363342,...,NaN,19.381405,20.741724,NaN,6.276816,NaN,NaN,20.4,1.030,NaN
1,2013-03-07,NaN,43.194801,134.233261,17.437082,NaN,NaN,NaN,NaN,33.174763,...,NaN,19.560863,21.486933,NaN,6.147664,NaN,NaN,20.5,1.050,0.099423
2,2013-03-08,NaN,43.123638,134.141327,17.212570,NaN,NaN,NaN,NaN,34.861614,...,NaN,19.471134,20.990126,NaN,6.199325,NaN,NaN,20.6,0.970,1.377876
3,2013-03-11,NaN,42.910152,134.049377,17.137733,NaN,NaN,NaN,NaN,35.705040,...,NaN,19.471134,21.362732,NaN,6.302645,NaN,NaN,20.6,1.050,0.467752
4,2013-03-12,NaN,43.692932,133.497757,17.212570,NaN,NaN,NaN,NaN,35.002186,...,NaN,19.201946,20.865921,NaN,6.354307,NaN,NaN,20.6,1.035,-1.992477


In [11]:
for column in df_price_2013.columns[1:-1]:  # Exclude the 'Date' and '^OMXSPI' column

    stock_returns = ARs_df[column].pct_change()

    result_df = pd.DataFrame({
    'Date': ARs_df['Date'],
    'Stock_Returns': stock_returns,
    'Market_Returns': ARs_df['market_returns']})

    result_df['beta'] = result_df['Stock_Returns'].rolling(window=window_size).cov(result_df['Market_Returns']).div(result_df['Market_Returns'].rolling(window=window_size).var())
    result_df = pd.merge(result_df,ARs_df[['Date','Swedish Treasury Bills (SE TB 1 Month)']], left_on='Date',right_on='Date', how='left')

    # Ri = Rf + beta * (Rm-Rf) + ei --> Ri - [Rf + beta * (Rm - Rf)]
    result_df['MMAR'] = result_df['Swedish Treasury Bills (SE TB 1 Month)']+ result_df['beta'] * (result_df['Market_Returns'] - result_df['Swedish Treasury Bills (SE TB 1 Month)'])

    ARs_df[f'{column}_ARs'] = result_df['Stock_Returns'] - result_df['MMAR']
    ARs_df.drop(columns=f'{column}',inplace=True)


### **Proxy A :**

Abnormal stock returns following large stock price increases and decreases by market sign and market capitalization.

In [12]:
proxy_a_df = pd.read_excel(f'{path}/raw_data/main/proxy_a.xlsx')
df_mid = pd.read_excel (f'{path}/raw_data/main/mid_caps.xlsx')
total_a_df = pd.merge(proxy_a_df,ARs_df,on='Date',how='left')


In [13]:
total_a_df.head()


,Unnamed: 0,Date,8TRA.ST_Increase_small_thres,8TRA.ST_Decrease_small_thres,8TRA.ST_Increase_large_thres,8TRA.ST_Decrease_large_thres,AAK.ST_Increase_small_thres,AAK.ST_Decrease_small_thres,AAK.ST_Increase_large_thres,AAK.ST_Decrease_large_thres,...,VPLAY-B.ST_ARs,WALL-B.ST_ARs,WBGR-B.ST_ARs,WIHL.ST_ARs,WISE.ST_ARs,WTW-A.ST_ARs,XANO-B.ST_ARs,XBRANE.ST_ARs,XSPRAY.ST_ARs,XVIVO.ST_ARs
0,0,2014-03-06,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2014-03-07,0,0,0,0,0,0,0,0,...,NaN,0.284278,NaN,1.712153,2.648586,NaN,0.123482,NaN,NaN,-10.518450
2,2,2014-03-10,0,0,0,0,0,0,0,0,...,NaN,0.551437,NaN,-1.087446,1.543361,NaN,-2.931026,NaN,NaN,-4.078193
3,3,2014-03-11,0,0,0,0,0,0,0,0,...,NaN,2.285070,NaN,-1.003612,5.536781,NaN,-0.285982,NaN,NaN,3.144449
4,4,2014-03-12,0,0,0,0,0,0,0,0,...,NaN,-2.563473,NaN,-1.095838,-1.455698,NaN,6.127125,NaN,NaN,10.897615


#### Price increases with market increases: 

##### Testing events individually: 

In [14]:

# Create a dictionary to store the result for each threshold
results_dict_a_eight_increase_m = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'P_Value_1_Day': [],
                'Avg_Next_2_Days_Returns': [],
                'P_Value_2_Days': [],
                'Avg_Next_5_Days_Returns': [],
                'P_Value_5_Days': [],
                'Avg_Next_20_Days_Returns': [],
                'P_Value_20_Days': []}

results_dict_a_ten_increase_m = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'P_Value_1_Day': [],
                'Avg_Next_2_Days_Returns': [],
                'P_Value_2_Days': [],
                'Avg_Next_5_Days_Returns': [],
                'P_Value_5_Days': [],
                'Avg_Next_20_Days_Returns': [],
                'P_Value_20_Days': []}

# Calculate average ARs and p-values for 8% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Increase_small_thres'] == 1) & (total_a_df['Market_Return_Increase'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1
        idx_event_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0]

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

        # Average_next_day
            avg_next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            p_value_1_day = ttest_1samp(avg_next_day_returns,0, alternative='two-sided').pvalue


            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()
            p_value_2_days = ttest_1samp(next_2_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()
            p_value_5_days = ttest_1samp(next_5_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()
            p_value_20_days = ttest_1samp(next_20_days_returns, 0, alternative='two-sided').pvalue


        results_dict_a_eight_increase_m['Company'].append(i)
        results_dict_a_eight_increase_m['Event_Date'].append(event_date)
        results_dict_a_eight_increase_m['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_eight_increase_m['P_Value_1_Day'].append(p_value_1_day)
        results_dict_a_eight_increase_m['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_eight_increase_m['P_Value_2_Days'].append(p_value_2_days)
        results_dict_a_eight_increase_m['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_eight_increase_m['P_Value_5_Days'].append(p_value_5_days)
        results_dict_a_eight_increase_m['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)
        results_dict_a_eight_increase_m['P_Value_20_Days'].append(p_value_20_days)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns} , P value: {p_value_1_day} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns},P value: {p_value_2_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns},P value: {p_value_5_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns},P value: {p_value_20_days} ')

results_significance_a_eight_increase_m = pd.DataFrame(results_dict_a_eight_increase_m)


# Calculate average ARs and p-values for 10% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Increase_large_thres'] == 1) & (total_a_df['Market_Return_Increase'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1
        idx_event_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0]

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

        # Average_next_day
            avg_next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            p_value_1_day = ttest_1samp(avg_next_day_returns,0, alternative='two-sided').pvalue


            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()
            p_value_2_days = ttest_1samp(next_2_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()
            p_value_5_days = ttest_1samp(next_5_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()
            p_value_20_days = ttest_1samp(next_20_days_returns, 0, alternative='two-sided').pvalue

        results_dict_a_ten_increase_m['Company'].append(i)
        results_dict_a_ten_increase_m['Event_Date'].append(event_date)
        results_dict_a_ten_increase_m['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_ten_increase_m['P_Value_1_Day'].append(p_value_1_day)
        results_dict_a_ten_increase_m['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_ten_increase_m['P_Value_2_Days'].append(p_value_2_days)
        results_dict_a_ten_increase_m['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_ten_increase_m['P_Value_5_Days'].append(p_value_5_days)
        results_dict_a_ten_increase_m['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)
        results_dict_a_ten_increase_m['P_Value_20_Days'].append(p_value_20_days)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns} , P value: {p_value_1_day} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns},P value: {p_value_2_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns},P value: {p_value_5_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns},P value: {p_value_20_days} ')

results_significance_a_ten_increase_m = pd.DataFrame(results_dict_a_ten_increase_m)
print(results_significance_a_eight_increase_m.head())
print(results_significance_a_ten_increase_m.head())


Company: ACAD.ST, Date: 2020-05-05 00:00:00, Avg Next Day Returns: 10.717521199479394 , P value: nan 
Company: ACAD.ST, Date: 2020-05-05 00:00:00, Avg Next 2 Days Returns: 5.186195742669233,P value: 0.520493591291451 
Company: ACAD.ST, Date: 2020-05-05 00:00:00, Avg Next 5 Days Returns: 0.7662937054564584,P value: 0.8020367962379538 
Company: ACAD.ST, Date: 2020-05-05 00:00:00, Avg Next 20 Days Returns: 0.7252657612623356,P value: 0.6399207551749436 
Company: ACAD.ST, Date: 2021-06-01 00:00:00, Avg Next Day Returns: -17.37933473325844 , P value: nan 
Company: ACAD.ST, Date: 2021-06-01 00:00:00, Avg Next 2 Days Returns: -10.552588391521239,P value: 0.36555431169945124 
Company: ACAD.ST, Date: 2021-06-01 00:00:00, Avg Next 5 Days Returns: -4.718604937029271,P value: 0.22577365497555452 
Company: ACAD.ST, Date: 2021-06-01 00:00:00, Avg Next 20 Days Returns: -0.956160883379815,P value: 0.43045486421025003 
Company: ACAD.ST, Date: 2022-09-12 00:00:00, Avg Next Day Returns: 1.221740757331894

##### Testing events aggregately: 

In [15]:
# Create a dictionary to store the result for threshold 8%
results_dict_a_eight_increase_alt_m = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'Avg_Next_2_Days_Returns': [],
                'Avg_Next_5_Days_Returns': [],
                'Avg_Next_20_Days_Returns': []}
Next_Day_Returns=[]
Next_2_Days_Returns=[]
Next_5_Days_Returns=[]
Next_20_Days_Returns=[]


# Calculate average ARs and p-values for 8% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Increase_small_thres'] == 1) & (total_a_df['Market_Return_Increase'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']
        list_2=next_2_days_returns.dropna().to_list()
        Next_2_Days_Returns.extend(list_2)

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']
        list_5=next_5_days_returns.dropna().to_list()
        Next_5_Days_Returns.extend(list_5)

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']
        list_20=next_20_days_returns.dropna().to_list()
        Next_20_Days_Returns.extend(list_20)


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

            # Average_next_day
            next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            Next_Day_Returns.append(next_day_returns)

            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()


        results_dict_a_eight_increase_alt_m['Company'].append(i)
        results_dict_a_eight_increase_alt_m['Event_Date'].append(event_date)
        results_dict_a_eight_increase_alt_m['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_eight_increase_alt_m['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_eight_increase_alt_m['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_eight_increase_alt_m['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns}')

results_significance_a_eight_increase_alt_m = pd.DataFrame(results_dict_a_eight_increase_alt_m)

results_significance_a_eight_increase_alt_result_m = {'Window':['1','1-2','1-5','1-20'],
                                                    'Avg_ARs_%':[],
                                                    'p_value':[]}
results_significance_a_eight_increase_alt_result_m['Avg_ARs_%'].append(results_significance_a_eight_increase_alt_m['Avg_Next_Day_Returns'].mean())
results_significance_a_eight_increase_alt_result_m['Avg_ARs_%'].append(results_significance_a_eight_increase_alt_m['Avg_Next_2_Days_Returns'].mean())
results_significance_a_eight_increase_alt_result_m['Avg_ARs_%'].append(results_significance_a_eight_increase_alt_m['Avg_Next_5_Days_Returns'].mean())
results_significance_a_eight_increase_alt_result_m['Avg_ARs_%'].append(results_significance_a_eight_increase_alt_m['Avg_Next_20_Days_Returns'].mean())

result_1=ttest_1samp(Next_Day_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_increase_alt_result_m['p_value'].append(result_1)

result_2=ttest_1samp(Next_2_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_increase_alt_result_m['p_value'].append(result_2)
result_5=ttest_1samp(Next_5_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_increase_alt_result_m['p_value'].append(result_5)
result_20=ttest_1samp(Next_20_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_increase_alt_result_m['p_value'].append(result_20)
results_significance_a_eight_increase_alt_result_m=pd.DataFrame(results_significance_a_eight_increase_alt_result_m)

# Create a dictionary to store the result for threshold 10%

results_dict_a_ten_increase_alt_m = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'Avg_Next_2_Days_Returns': [],
                'Avg_Next_5_Days_Returns': [],
                'Avg_Next_20_Days_Returns': []}
Next_Day_Returns=[]
Next_2_Days_Returns=[]
Next_5_Days_Returns=[]
Next_20_Days_Returns=[]


# Calculate average ARs and p-values for 10% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Increase_large_thres'] == 1) & (total_a_df['Market_Return_Increase'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']
        list_2=next_2_days_returns.dropna().to_list()
        Next_2_Days_Returns.extend(list_2)

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']
        list_5=next_5_days_returns.dropna().to_list()
        Next_5_Days_Returns.extend(list_5)

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']
        list_20=next_20_days_returns.dropna().to_list()
        Next_20_Days_Returns.extend(list_20)


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

            # Average_next_day
            next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            Next_Day_Returns.append(next_day_returns)

            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()


        results_dict_a_ten_increase_alt_m['Company'].append(i)
        results_dict_a_ten_increase_alt_m['Event_Date'].append(event_date)
        results_dict_a_ten_increase_alt_m['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_ten_increase_alt_m['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_ten_increase_alt_m['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_ten_increase_alt_m['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns}')

results_significance_a_ten_increase_alt_m = pd.DataFrame(results_dict_a_ten_increase_alt_m)

results_significance_a_ten_increase_alt_result_m = {'Window':['1','1-2','1-5','1-20'],
                                                    'Avg_ARs_%':[],
                                                    'p_value':[]}
results_significance_a_ten_increase_alt_result_m['Avg_ARs_%'].append(results_significance_a_ten_increase_alt_m['Avg_Next_Day_Returns'].mean())
results_significance_a_ten_increase_alt_result_m['Avg_ARs_%'].append(results_significance_a_ten_increase_alt_m['Avg_Next_2_Days_Returns'].mean())
results_significance_a_ten_increase_alt_result_m['Avg_ARs_%'].append(results_significance_a_ten_increase_alt_m['Avg_Next_5_Days_Returns'].mean())
results_significance_a_ten_increase_alt_result_m['Avg_ARs_%'].append(results_significance_a_ten_increase_alt_m['Avg_Next_20_Days_Returns'].mean())

result_1=ttest_1samp(Next_Day_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_increase_alt_result_m['p_value'].append(result_1)

result_2=ttest_1samp(Next_2_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_increase_alt_result_m['p_value'].append(result_2)
result_5=ttest_1samp(Next_5_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_increase_alt_result_m['p_value'].append(result_5)
result_20=ttest_1samp(Next_20_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_increase_alt_result_m['p_value'].append(result_20)
results_significance_a_ten_increase_alt_result_m=pd.DataFrame(results_significance_a_ten_increase_alt_result_m)

print(results_significance_a_eight_increase_alt_result_m.head())
print(results_significance_a_ten_increase_alt_result_m.head())


Company: ACAD.ST, Date: 2020-05-05 00:00:00, Avg Next Day Returns: 8.107344089646741
Company: ACAD.ST, Date: 2020-05-05 00:00:00, Avg Next 2 Days Returns: 5.186195742669233 
Company: ACAD.ST, Date: 2020-05-05 00:00:00, Avg Next 5 Days Returns: 0.7662937054564584
Company: ACAD.ST, Date: 2020-05-05 00:00:00, Avg Next 20 Days Returns: 0.7252657612623356
Company: ACAD.ST, Date: 2021-06-01 00:00:00, Avg Next Day Returns: 8.107344089646741
Company: ACAD.ST, Date: 2021-06-01 00:00:00, Avg Next 2 Days Returns: -10.552588391521239 
Company: ACAD.ST, Date: 2021-06-01 00:00:00, Avg Next 5 Days Returns: -4.718604937029271
Company: ACAD.ST, Date: 2021-06-01 00:00:00, Avg Next 20 Days Returns: -0.956160883379815
Company: ACAD.ST, Date: 2022-09-12 00:00:00, Avg Next Day Returns: 8.107344089646741
Company: ACAD.ST, Date: 2022-09-12 00:00:00, Avg Next 2 Days Returns: 1.60653577081271 
Company: ACAD.ST, Date: 2022-09-12 00:00:00, Avg Next 5 Days Returns: 0.7435136465614769
Company: ACAD.ST, Date: 2022-0

#### Price increases with market decreases: 

##### Testing events individually: 

In [16]:

# Create a dictionary to store the result for each threshold
results_dict_a_eight_increase_m_dec = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'P_Value_1_Day': [],
                'Avg_Next_2_Days_Returns': [],
                'P_Value_2_Days': [],
                'Avg_Next_5_Days_Returns': [],
                'P_Value_5_Days': [],
                'Avg_Next_20_Days_Returns': [],
                'P_Value_20_Days': []}

results_dict_a_ten_increase_m_dec = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'P_Value_1_Day': [],
                'Avg_Next_2_Days_Returns': [],
                'P_Value_2_Days': [],
                'Avg_Next_5_Days_Returns': [],
                'P_Value_5_Days': [],
                'Avg_Next_20_Days_Returns': [],
                'P_Value_20_Days': []}

# Calculate average ARs and p-values for 8% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Increase_small_thres'] == 1) & (total_a_df['Market_Return_Decrease'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1
        idx_event_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0]

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

        # Average_next_day
            avg_next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            p_value_1_day = ttest_1samp(avg_next_day_returns,0, alternative='two-sided').pvalue


            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()
            p_value_2_days = ttest_1samp(next_2_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()
            p_value_5_days = ttest_1samp(next_5_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()
            p_value_20_days = ttest_1samp(next_20_days_returns, 0, alternative='two-sided').pvalue


        results_dict_a_eight_increase_m_dec['Company'].append(i)
        results_dict_a_eight_increase_m_dec['Event_Date'].append(event_date)
        results_dict_a_eight_increase_m_dec['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_eight_increase_m_dec['P_Value_1_Day'].append(p_value_1_day)
        results_dict_a_eight_increase_m_dec['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_eight_increase_m_dec['P_Value_2_Days'].append(p_value_2_days)
        results_dict_a_eight_increase_m_dec['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_eight_increase_m_dec['P_Value_5_Days'].append(p_value_5_days)
        results_dict_a_eight_increase_m_dec['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)
        results_dict_a_eight_increase_m_dec['P_Value_20_Days'].append(p_value_20_days)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns} , P value: {p_value_1_day} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns},P value: {p_value_2_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns},P value: {p_value_5_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns},P value: {p_value_20_days} ')

results_significance_a_eight_increase_m_dec = pd.DataFrame(results_dict_a_eight_increase_m_dec)


# Calculate average ARs and p-values for 10% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Increase_large_thres'] == 1) & (total_a_df['Market_Return_Decrease'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1
        idx_event_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0]

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

        # Average_next_day
            avg_next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            p_value_1_day = ttest_1samp(avg_next_day_returns,0, alternative='two-sided').pvalue


            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()
            p_value_2_days = ttest_1samp(next_2_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()
            p_value_5_days = ttest_1samp(next_5_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()
            p_value_20_days = ttest_1samp(next_20_days_returns, 0, alternative='two-sided').pvalue

        results_dict_a_ten_increase_m_dec['Company'].append(i)
        results_dict_a_ten_increase_m_dec['Event_Date'].append(event_date)
        results_dict_a_ten_increase_m_dec['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_ten_increase_m_dec['P_Value_1_Day'].append(p_value_1_day)
        results_dict_a_ten_increase_m_dec['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_ten_increase_m_dec['P_Value_2_Days'].append(p_value_2_days)
        results_dict_a_ten_increase_m_dec['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_ten_increase_m_dec['P_Value_5_Days'].append(p_value_5_days)
        results_dict_a_ten_increase_m_dec['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)
        results_dict_a_ten_increase_m_dec['P_Value_20_Days'].append(p_value_20_days)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns} , P value: {p_value_1_day} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns},P value: {p_value_2_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns},P value: {p_value_5_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns},P value: {p_value_20_days} ')

results_significance_a_ten_increase_m_dec = pd.DataFrame(results_dict_a_ten_increase_m_dec)
print(results_significance_a_eight_increase_m_dec.head())
print(results_significance_a_ten_increase_m_dec.head())


Company: ACAD.ST, Date: 2020-01-31 00:00:00, Avg Next Day Returns: 7.460290117221901 , P value: nan 
Company: ACAD.ST, Date: 2020-01-31 00:00:00, Avg Next 2 Days Returns: 3.142098705092493,P value: 0.5995419967453051 
Company: ACAD.ST, Date: 2020-01-31 00:00:00, Avg Next 5 Days Returns: 0.706795316217113,P value: 0.7191280202390646 
Company: ACAD.ST, Date: 2020-01-31 00:00:00, Avg Next 20 Days Returns: -0.1977987118726808,P value: 0.8400891031479124 
Company: ALIG.ST, Date: 2020-03-20 00:00:00, Avg Next Day Returns: -8.180313783063106 , P value: nan 
Company: ALIG.ST, Date: 2020-03-20 00:00:00, Avg Next 2 Days Returns: -0.7213205744603748,P value: 0.9386266440821309 
Company: ALIG.ST, Date: 2020-03-20 00:00:00, Avg Next 5 Days Returns: -0.7542110369045251,P value: 0.8894375628656432 
Company: ALIG.ST, Date: 2020-03-20 00:00:00, Avg Next 20 Days Returns: 1.2939413943676437,P value: 0.45788441289501414 
Company: ALLIGO-B.ST, Date: 2022-03-23 00:00:00, Avg Next Day Returns: 26.51757257034

##### Testing events aggregately: 

In [17]:
# Create a dictionary to store the result for threshold 8%
results_dict_a_eight_increase_alt_m_dec = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'Avg_Next_2_Days_Returns': [],
                'Avg_Next_5_Days_Returns': [],
                'Avg_Next_20_Days_Returns': []}
Next_Day_Returns=[]
Next_2_Days_Returns=[]
Next_5_Days_Returns=[]
Next_20_Days_Returns=[]


# Calculate average ARs and p-values for 8% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Increase_small_thres'] == 1) & (total_a_df['Market_Return_Decrease'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']
        list_2=next_2_days_returns.dropna().to_list()
        Next_2_Days_Returns.extend(list_2)

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']
        list_5=next_5_days_returns.dropna().to_list()
        Next_5_Days_Returns.extend(list_5)

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']
        list_20=next_20_days_returns.dropna().to_list()
        Next_20_Days_Returns.extend(list_20)


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

            # Average_next_day
            next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            Next_Day_Returns.append(next_day_returns)

            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()


        results_dict_a_eight_increase_alt_m_dec['Company'].append(i)
        results_dict_a_eight_increase_alt_m_dec['Event_Date'].append(event_date)
        results_dict_a_eight_increase_alt_m_dec['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_eight_increase_alt_m_dec['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_eight_increase_alt_m_dec['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_eight_increase_alt_m_dec['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns}')

results_significance_a_eight_increase_alt_m_dec = pd.DataFrame(results_dict_a_eight_increase_alt_m_dec)

results_significance_a_eight_increase_alt_result_m_dec = {'Window':['1','1-2','1-5','1-20'],
                                                    'Avg_ARs_%':[],
                                                    'p_value':[]}
results_significance_a_eight_increase_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_eight_increase_alt_m_dec['Avg_Next_Day_Returns'].mean())
results_significance_a_eight_increase_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_eight_increase_alt_m_dec['Avg_Next_2_Days_Returns'].mean())
results_significance_a_eight_increase_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_eight_increase_alt_m_dec['Avg_Next_5_Days_Returns'].mean())
results_significance_a_eight_increase_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_eight_increase_alt_m_dec['Avg_Next_20_Days_Returns'].mean())

result_1=ttest_1samp(Next_Day_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_increase_alt_result_m_dec['p_value'].append(result_1)

result_2=ttest_1samp(Next_2_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_increase_alt_result_m_dec['p_value'].append(result_2)
result_5=ttest_1samp(Next_5_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_increase_alt_result_m_dec['p_value'].append(result_5)
result_20=ttest_1samp(Next_20_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_increase_alt_result_m_dec['p_value'].append(result_20)
results_significance_a_eight_increase_alt_result_m_dec=pd.DataFrame(results_significance_a_eight_increase_alt_result_m_dec)

# Create a dictionary to store the result for threshold 10%

results_dict_a_ten_increase_alt_m_dec = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'Avg_Next_2_Days_Returns': [],
                'Avg_Next_5_Days_Returns': [],
                'Avg_Next_20_Days_Returns': []}
Next_Day_Returns=[]
Next_2_Days_Returns=[]
Next_5_Days_Returns=[]
Next_20_Days_Returns=[]


# Calculate average ARs and p-values for 10% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Increase_large_thres'] == 1) & (total_a_df['Market_Return_Decrease'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']
        list_2=next_2_days_returns.dropna().to_list()
        Next_2_Days_Returns.extend(list_2)

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']
        list_5=next_5_days_returns.dropna().to_list()
        Next_5_Days_Returns.extend(list_5)

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']
        list_20=next_20_days_returns.dropna().to_list()
        Next_20_Days_Returns.extend(list_20)


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

            # Average_next_day
            next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            Next_Day_Returns.append(next_day_returns)

            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()


        results_dict_a_ten_increase_alt_m_dec['Company'].append(i)
        results_dict_a_ten_increase_alt_m_dec['Event_Date'].append(event_date)
        results_dict_a_ten_increase_alt_m_dec['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_ten_increase_alt_m_dec['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_ten_increase_alt_m_dec['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_ten_increase_alt_m_dec['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns}')

results_significance_a_ten_increase_alt_m_dec = pd.DataFrame(results_dict_a_ten_increase_alt_m_dec)

results_significance_a_ten_increase_alt_result_m_dec = {'Window':['1','1-2','1-5','1-20'],
                                                    'Avg_ARs_%':[],
                                                    'p_value':[]}
results_significance_a_ten_increase_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_ten_increase_alt_m_dec['Avg_Next_Day_Returns'].mean())
results_significance_a_ten_increase_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_ten_increase_alt_m_dec['Avg_Next_2_Days_Returns'].mean())
results_significance_a_ten_increase_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_ten_increase_alt_m_dec['Avg_Next_5_Days_Returns'].mean())
results_significance_a_ten_increase_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_ten_increase_alt_m_dec['Avg_Next_20_Days_Returns'].mean())

result_1=ttest_1samp(Next_Day_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_increase_alt_result_m_dec['p_value'].append(result_1)

result_2=ttest_1samp(Next_2_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_increase_alt_result_m_dec['p_value'].append(result_2)
result_5=ttest_1samp(Next_5_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_increase_alt_result_m_dec['p_value'].append(result_5)
result_20=ttest_1samp(Next_20_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_increase_alt_result_m_dec['p_value'].append(result_20)
results_significance_a_ten_increase_alt_result_m_dec=pd.DataFrame(results_significance_a_ten_increase_alt_result_m)

print(results_significance_a_eight_increase_alt_result_m_dec.head())
print(results_significance_a_ten_increase_alt_result_m_dec.head())


Company: ACAD.ST, Date: 2020-01-31 00:00:00, Avg Next Day Returns: -27.65859327696292
Company: ACAD.ST, Date: 2020-01-31 00:00:00, Avg Next 2 Days Returns: 3.142098705092493 
Company: ACAD.ST, Date: 2020-01-31 00:00:00, Avg Next 5 Days Returns: 0.706795316217113
Company: ACAD.ST, Date: 2020-01-31 00:00:00, Avg Next 20 Days Returns: -0.1977987118726808
Company: ALIG.ST, Date: 2020-03-20 00:00:00, Avg Next Day Returns: -27.65859327696292
Company: ALIG.ST, Date: 2020-03-20 00:00:00, Avg Next 2 Days Returns: -0.7213205744603748 
Company: ALIG.ST, Date: 2020-03-20 00:00:00, Avg Next 5 Days Returns: -0.7542110369045251
Company: ALIG.ST, Date: 2020-03-20 00:00:00, Avg Next 20 Days Returns: 1.2939413943676437
Company: ALLIGO-B.ST, Date: 2022-03-23 00:00:00, Avg Next Day Returns: -27.65859327696292
Company: ALLIGO-B.ST, Date: 2022-03-23 00:00:00, Avg Next 2 Days Returns: 8.586548069382713 
Company: ALLIGO-B.ST, Date: 2022-03-23 00:00:00, Avg Next 5 Days Returns: 3.002754509862436
Company: ALLIG

#### Price decreases with market increases: 

##### Testing events individually: 

In [18]:

# Create a dictionary to store the result for each threshold
results_dict_a_eight_decrease_m = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'P_Value_1_Day': [],
                'Avg_Next_2_Days_Returns': [],
                'P_Value_2_Days': [],
                'Avg_Next_5_Days_Returns': [],
                'P_Value_5_Days': [],
                'Avg_Next_20_Days_Returns': [],
                'P_Value_20_Days': []}

results_dict_a_ten_decrease_m = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'P_Value_1_Day': [],
                'Avg_Next_2_Days_Returns': [],
                'P_Value_2_Days': [],
                'Avg_Next_5_Days_Returns': [],
                'P_Value_5_Days': [],
                'Avg_Next_20_Days_Returns': [],
                'P_Value_20_Days': []}

# Calculate average ARs and p-values for 8% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Decrease_small_thres'] == 1) & (total_a_df['Market_Return_Increase'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1
        idx_event_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0]

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

        # Average_next_day
            avg_next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            p_value_1_day = ttest_1samp(avg_next_day_returns,0, alternative='two-sided').pvalue


            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()
            p_value_2_days = ttest_1samp(next_2_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()
            p_value_5_days = ttest_1samp(next_5_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()
            p_value_20_days = ttest_1samp(next_20_days_returns, 0, alternative='two-sided').pvalue


        results_dict_a_eight_decrease_m['Company'].append(i)
        results_dict_a_eight_decrease_m['Event_Date'].append(event_date)
        results_dict_a_eight_decrease_m['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_eight_decrease_m['P_Value_1_Day'].append(p_value_1_day)
        results_dict_a_eight_decrease_m['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_eight_decrease_m['P_Value_2_Days'].append(p_value_2_days)
        results_dict_a_eight_decrease_m['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_eight_decrease_m['P_Value_5_Days'].append(p_value_5_days)
        results_dict_a_eight_decrease_m['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)
        results_dict_a_eight_decrease_m['P_Value_20_Days'].append(p_value_20_days)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns} , P value: {p_value_1_day} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns},P value: {p_value_2_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns},P value: {p_value_5_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns},P value: {p_value_20_days} ')

results_significance_a_eight_decrease_m = pd.DataFrame(results_dict_a_eight_decrease_m)


# Calculate average ARs and p-values for 10% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Decrease_large_thres'] == 1) & (total_a_df['Market_Return_Increase'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1
        idx_event_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0]

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

        # Average_next_day
            avg_next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            p_value_1_day = ttest_1samp(avg_next_day_returns,0, alternative='two-sided').pvalue


            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()
            p_value_2_days = ttest_1samp(next_2_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()
            p_value_5_days = ttest_1samp(next_5_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()
            p_value_20_days = ttest_1samp(next_20_days_returns, 0, alternative='two-sided').pvalue

        results_dict_a_ten_decrease_m['Company'].append(i)
        results_dict_a_ten_decrease_m['Event_Date'].append(event_date)
        results_dict_a_ten_decrease_m['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_ten_decrease_m['P_Value_1_Day'].append(p_value_1_day)
        results_dict_a_ten_decrease_m['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_ten_decrease_m['P_Value_2_Days'].append(p_value_2_days)
        results_dict_a_ten_decrease_m['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_ten_decrease_m['P_Value_5_Days'].append(p_value_5_days)
        results_dict_a_ten_decrease_m['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)
        results_dict_a_ten_decrease_m['P_Value_20_Days'].append(p_value_20_days)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns} , P value: {p_value_1_day} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns},P value: {p_value_2_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns},P value: {p_value_5_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns},P value: {p_value_20_days} ')

results_significance_a_ten_decrease_m = pd.DataFrame(results_dict_a_ten_decrease_m)
print(results_significance_a_eight_decrease_m.head())
print(results_significance_a_ten_decrease_m.head())


Company: ALIG.ST, Date: 2018-08-21 00:00:00, Avg Next Day Returns: 12.189960864091415 , P value: nan 
Company: ALIG.ST, Date: 2018-08-21 00:00:00, Avg Next 2 Days Returns: 3.431859011212362,P value: 0.7622476524885579 
Company: ALIG.ST, Date: 2018-08-21 00:00:00, Avg Next 5 Days Returns: 0.12659603813839437,P value: 0.9694108331346938 
Company: ALIG.ST, Date: 2018-08-21 00:00:00, Avg Next 20 Days Returns: 0.25143750816415933,P value: 0.8002502029432765 
Company: AOI.ST, Date: 2014-10-23 00:00:00, Avg Next Day Returns: -13.099843521522438 , P value: nan 
Company: AOI.ST, Date: 2014-10-23 00:00:00, Avg Next 2 Days Returns: -4.759760676094668,P value: 0.6698467385958751 
Company: AOI.ST, Date: 2014-10-23 00:00:00, Avg Next 5 Days Returns: -2.4646894981647156,P value: 0.4785845627763018 
Company: AOI.ST, Date: 2014-10-23 00:00:00, Avg Next 20 Days Returns: -1.2594398245706746,P value: 0.4148734524236475 
Company: AQ.ST, Date: 2018-02-22 00:00:00, Avg Next Day Returns: -6.166671845856133 , 

##### Testing events aggregately: 

In [19]:
# Create a dictionary to store the result for threshold 8%
results_dict_a_eight_decrease_alt_m = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'Avg_Next_2_Days_Returns': [],
                'Avg_Next_5_Days_Returns': [],
                'Avg_Next_20_Days_Returns': []}
Next_Day_Returns=[]
Next_2_Days_Returns=[]
Next_5_Days_Returns=[]
Next_20_Days_Returns=[]


# Calculate average ARs and p-values for 8% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Decrease_small_thres'] == 1) & (total_a_df['Market_Return_Increase'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']
        list_2=next_2_days_returns.dropna().to_list()
        Next_2_Days_Returns.extend(list_2)

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']
        list_5=next_5_days_returns.dropna().to_list()
        Next_5_Days_Returns.extend(list_5)

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']
        list_20=next_20_days_returns.dropna().to_list()
        Next_20_Days_Returns.extend(list_20)


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

            # Average_next_day
            next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            Next_Day_Returns.append(next_day_returns)

            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()


        results_dict_a_eight_decrease_alt_m['Company'].append(i)
        results_dict_a_eight_decrease_alt_m['Event_Date'].append(event_date)
        results_dict_a_eight_decrease_alt_m['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_eight_decrease_alt_m['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_eight_decrease_alt_m['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_eight_decrease_alt_m['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns}')

results_significance_a_eight_decrease_alt_m = pd.DataFrame(results_dict_a_eight_decrease_alt_m)

results_significance_a_eight_decrease_alt_result_m = {'Window':['1','1-2','1-5','1-20'],
                                                    'Avg_ARs_%':[],
                                                    'p_value':[]}
results_significance_a_eight_decrease_alt_result_m['Avg_ARs_%'].append(results_significance_a_eight_decrease_alt_m['Avg_Next_Day_Returns'].mean())
results_significance_a_eight_decrease_alt_result_m['Avg_ARs_%'].append(results_significance_a_eight_decrease_alt_m['Avg_Next_2_Days_Returns'].mean())
results_significance_a_eight_decrease_alt_result_m['Avg_ARs_%'].append(results_significance_a_eight_decrease_alt_m['Avg_Next_5_Days_Returns'].mean())
results_significance_a_eight_decrease_alt_result_m['Avg_ARs_%'].append(results_significance_a_eight_decrease_alt_m['Avg_Next_20_Days_Returns'].mean())

result_1=ttest_1samp(Next_Day_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_decrease_alt_result_m['p_value'].append(result_1)

result_2=ttest_1samp(Next_2_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_decrease_alt_result_m['p_value'].append(result_2)
result_5=ttest_1samp(Next_5_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_decrease_alt_result_m['p_value'].append(result_5)
result_20=ttest_1samp(Next_20_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_decrease_alt_result_m['p_value'].append(result_20)
results_significance_a_eight_decrease_alt_result_m=pd.DataFrame(results_significance_a_eight_decrease_alt_result_m)

# Create a dictionary to store the result for threshold 10%

results_dict_a_ten_decrease_alt_m = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'Avg_Next_2_Days_Returns': [],
                'Avg_Next_5_Days_Returns': [],
                'Avg_Next_20_Days_Returns': []}
Next_Day_Returns=[]
Next_2_Days_Returns=[]
Next_5_Days_Returns=[]
Next_20_Days_Returns=[]


# Calculate average ARs and p-values for 10% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Decrease_large_thres'] == 1) & (total_a_df['Market_Return_Increase'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']
        list_2=next_2_days_returns.dropna().to_list()
        Next_2_Days_Returns.extend(list_2)

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']
        list_5=next_5_days_returns.dropna().to_list()
        Next_5_Days_Returns.extend(list_5)

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']
        list_20=next_20_days_returns.dropna().to_list()
        Next_20_Days_Returns.extend(list_20)


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

            # Average_next_day
            next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            Next_Day_Returns.append(next_day_returns)

            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()


        results_dict_a_ten_decrease_alt_m['Company'].append(i)
        results_dict_a_ten_decrease_alt_m['Event_Date'].append(event_date)
        results_dict_a_ten_decrease_alt_m['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_ten_decrease_alt_m['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_ten_decrease_alt_m['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_ten_decrease_alt_m['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns}')

results_significance_a_ten_decrease_alt_m = pd.DataFrame(results_dict_a_ten_decrease_alt_m)

results_significance_a_ten_decrease_alt_result_m = {'Window':['1','1-2','1-5','1-20'],
                                                    'Avg_ARs_%':[],
                                                    'p_value':[]}
results_significance_a_ten_decrease_alt_result_m['Avg_ARs_%'].append(results_significance_a_ten_decrease_alt_m['Avg_Next_Day_Returns'].mean())
results_significance_a_ten_decrease_alt_result_m['Avg_ARs_%'].append(results_significance_a_ten_decrease_alt_m['Avg_Next_2_Days_Returns'].mean())
results_significance_a_ten_decrease_alt_result_m['Avg_ARs_%'].append(results_significance_a_ten_decrease_alt_m['Avg_Next_5_Days_Returns'].mean())
results_significance_a_ten_decrease_alt_result_m['Avg_ARs_%'].append(results_significance_a_ten_decrease_alt_m['Avg_Next_20_Days_Returns'].mean())

result_1=ttest_1samp(Next_Day_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_decrease_alt_result_m['p_value'].append(result_1)

result_2=ttest_1samp(Next_2_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_decrease_alt_result_m['p_value'].append(result_2)
result_5=ttest_1samp(Next_5_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_decrease_alt_result_m['p_value'].append(result_5)
result_20=ttest_1samp(Next_20_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_decrease_alt_result_m['p_value'].append(result_20)
results_significance_a_ten_decrease_alt_result_m=pd.DataFrame(results_significance_a_ten_decrease_alt_result_m)

print(results_significance_a_eight_decrease_alt_result_m.head())
print(results_significance_a_ten_decrease_alt_result_m.head())


Company: ALIG.ST, Date: 2018-08-21 00:00:00, Avg Next Day Returns: 6.434761408954161
Company: ALIG.ST, Date: 2018-08-21 00:00:00, Avg Next 2 Days Returns: 3.431859011212362 
Company: ALIG.ST, Date: 2018-08-21 00:00:00, Avg Next 5 Days Returns: 0.12659603813839437
Company: ALIG.ST, Date: 2018-08-21 00:00:00, Avg Next 20 Days Returns: 0.25143750816415933
Company: AOI.ST, Date: 2014-10-23 00:00:00, Avg Next Day Returns: 6.434761408954161
Company: AOI.ST, Date: 2014-10-23 00:00:00, Avg Next 2 Days Returns: -4.759760676094668 
Company: AOI.ST, Date: 2014-10-23 00:00:00, Avg Next 5 Days Returns: -2.4646894981647156
Company: AOI.ST, Date: 2014-10-23 00:00:00, Avg Next 20 Days Returns: -1.2594398245706746
Company: AQ.ST, Date: 2018-02-22 00:00:00, Avg Next Day Returns: 6.434761408954161
Company: AQ.ST, Date: 2018-02-22 00:00:00, Avg Next 2 Days Returns: -9.164258814273156 
Company: AQ.ST, Date: 2018-02-22 00:00:00, Avg Next 5 Days Returns: -4.240838556099169
Company: AQ.ST, Date: 2018-02-22 00

#### Price decreases with market decreases: 

##### Testing events individually: 

In [20]:

# Create a dictionary to store the result for each threshold
results_dict_a_eight_decrease_m_dec = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'P_Value_1_Day': [],
                'Avg_Next_2_Days_Returns': [],
                'P_Value_2_Days': [],
                'Avg_Next_5_Days_Returns': [],
                'P_Value_5_Days': [],
                'Avg_Next_20_Days_Returns': [],
                'P_Value_20_Days': []}

results_dict_a_ten_decrease_m_dec = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'P_Value_1_Day': [],
                'Avg_Next_2_Days_Returns': [],
                'P_Value_2_Days': [],
                'Avg_Next_5_Days_Returns': [],
                'P_Value_5_Days': [],
                'Avg_Next_20_Days_Returns': [],
                'P_Value_20_Days': []}

# Calculate average ARs and p-values for 8% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Decrease_small_thres'] == 1) & (total_a_df['Market_Return_Decrease'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1
        idx_event_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0]

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

        # Average_next_day
            avg_next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            p_value_1_day = ttest_1samp(avg_next_day_returns,0, alternative='two-sided').pvalue


            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()
            p_value_2_days = ttest_1samp(next_2_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()
            p_value_5_days = ttest_1samp(next_5_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()
            p_value_20_days = ttest_1samp(next_20_days_returns, 0, alternative='two-sided').pvalue


        results_dict_a_eight_decrease_m_dec['Company'].append(i)
        results_dict_a_eight_decrease_m_dec['Event_Date'].append(event_date)
        results_dict_a_eight_decrease_m_dec['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_eight_decrease_m_dec['P_Value_1_Day'].append(p_value_1_day)
        results_dict_a_eight_decrease_m_dec['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_eight_decrease_m_dec['P_Value_2_Days'].append(p_value_2_days)
        results_dict_a_eight_decrease_m_dec['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_eight_decrease_m_dec['P_Value_5_Days'].append(p_value_5_days)
        results_dict_a_eight_decrease_m_dec['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)
        results_dict_a_eight_decrease_m_dec['P_Value_20_Days'].append(p_value_20_days)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns} , P value: {p_value_1_day} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns},P value: {p_value_2_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns},P value: {p_value_5_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns},P value: {p_value_20_days} ')

results_significance_a_eight_decrease_m_dec = pd.DataFrame(results_dict_a_eight_decrease_m_dec)


# Calculate average ARs and p-values for 10% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Decrease_large_thres'] == 1) & (total_a_df['Market_Return_Decrease'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1
        idx_event_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0]

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

        # Average_next_day
            avg_next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            p_value_1_day = ttest_1samp(avg_next_day_returns,0, alternative='two-sided').pvalue


            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()
            p_value_2_days = ttest_1samp(next_2_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()
            p_value_5_days = ttest_1samp(next_5_days_returns, 0, alternative='two-sided').pvalue

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()
            p_value_20_days = ttest_1samp(next_20_days_returns, 0, alternative='two-sided').pvalue

        results_dict_a_ten_decrease_m_dec['Company'].append(i)
        results_dict_a_ten_decrease_m_dec['Event_Date'].append(event_date)
        results_dict_a_ten_decrease_m_dec['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_ten_decrease_m_dec['P_Value_1_Day'].append(p_value_1_day)
        results_dict_a_ten_decrease_m_dec['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_ten_decrease_m_dec['P_Value_2_Days'].append(p_value_2_days)
        results_dict_a_ten_decrease_m_dec['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_ten_decrease_m_dec['P_Value_5_Days'].append(p_value_5_days)
        results_dict_a_ten_decrease_m_dec['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)
        results_dict_a_ten_decrease_m_dec['P_Value_20_Days'].append(p_value_20_days)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns} , P value: {p_value_1_day} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns},P value: {p_value_2_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns},P value: {p_value_5_days} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns},P value: {p_value_20_days} ')

results_significance_a_ten_decrease_m_dec = pd.DataFrame(results_dict_a_ten_decrease_m_dec)
print(results_significance_a_eight_decrease_m_dec.head())
print(results_significance_a_ten_decrease_m_dec.head())


Company: ACAD.ST, Date: 2020-03-12 00:00:00, Avg Next Day Returns: -6.699140994765537 , P value: nan 
Company: ACAD.ST, Date: 2020-03-12 00:00:00, Avg Next 2 Days Returns: -5.04436210958691,P value: 0.20179810842702714 
Company: ACAD.ST, Date: 2020-03-12 00:00:00, Avg Next 5 Days Returns: -0.44877507085113033,P value: 0.9338593460412136 
Company: ACAD.ST, Date: 2020-03-12 00:00:00, Avg Next 20 Days Returns: 1.352269727423933,P value: 0.4693323881449466 
Company: ALIG.ST, Date: 2020-03-16 00:00:00, Avg Next Day Returns: -8.31907158250291 , P value: nan 
Company: ALIG.ST, Date: 2020-03-16 00:00:00, Avg Next 2 Days Returns: -8.577213038153948,P value: 0.01915404956103847 
Company: ALIG.ST, Date: 2020-03-16 00:00:00, Avg Next 5 Days Returns: -0.508047797257214,P value: 0.9500142484800879 
Company: ALIG.ST, Date: 2020-03-16 00:00:00, Avg Next 20 Days Returns: 1.43486972176535,P value: 0.5510770637559751 
Company: ALLIGO-B.ST, Date: 2020-03-12 00:00:00, Avg Next Day Returns: 12.3993513298803

##### Testing events aggregately: 

In [21]:
# Create a dictionary to store the result for threshold 8%
results_dict_a_eight_decrease_alt_m_dec = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'Avg_Next_2_Days_Returns': [],
                'Avg_Next_5_Days_Returns': [],
                'Avg_Next_20_Days_Returns': []}
Next_Day_Returns=[]
Next_2_Days_Returns=[]
Next_5_Days_Returns=[]
Next_20_Days_Returns=[]


# Calculate average ARs and p-values for 8% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Decrease_small_thres'] == 1) & (total_a_df['Market_Return_Decrease'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']
        list_2=next_2_days_returns.dropna().to_list()
        Next_2_Days_Returns.extend(list_2)

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']
        list_5=next_5_days_returns.dropna().to_list()
        Next_5_Days_Returns.extend(list_5)

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']
        list_20=next_20_days_returns.dropna().to_list()
        Next_20_Days_Returns.extend(list_20)


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

            # Average_next_day
            next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            Next_Day_Returns.append(next_day_returns)

            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()


        results_dict_a_eight_decrease_alt_m_dec['Company'].append(i)
        results_dict_a_eight_decrease_alt_m_dec['Event_Date'].append(event_date)
        results_dict_a_eight_decrease_alt_m_dec['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_eight_decrease_alt_m_dec['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_eight_decrease_alt_m_dec['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_eight_decrease_alt_m_dec['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns}')

results_significance_a_eight_decrease_alt_m_dec = pd.DataFrame(results_dict_a_eight_decrease_alt_m_dec)

results_significance_a_eight_decrease_alt_result_m_dec = {'Window':['1','1-2','1-5','1-20'],
                                                    'Avg_ARs_%':[],
                                                    'p_value':[]}
results_significance_a_eight_decrease_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_eight_decrease_alt_m_dec['Avg_Next_Day_Returns'].mean())
results_significance_a_eight_decrease_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_eight_decrease_alt_m_dec['Avg_Next_2_Days_Returns'].mean())
results_significance_a_eight_decrease_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_eight_decrease_alt_m_dec['Avg_Next_5_Days_Returns'].mean())
results_significance_a_eight_decrease_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_eight_decrease_alt_m_dec['Avg_Next_20_Days_Returns'].mean())

result_1=ttest_1samp(Next_Day_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_decrease_alt_result_m_dec['p_value'].append(result_1)

result_2=ttest_1samp(Next_2_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_decrease_alt_result_m_dec['p_value'].append(result_2)
result_5=ttest_1samp(Next_5_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_decrease_alt_result_m_dec['p_value'].append(result_5)
result_20=ttest_1samp(Next_20_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_eight_decrease_alt_result_m_dec['p_value'].append(result_20)
results_significance_a_eight_decrease_alt_result_m_dec=pd.DataFrame(results_significance_a_eight_decrease_alt_result_m_dec)

# Create a dictionary to store the result for threshold 10%

results_dict_a_ten_decrease_alt_m_dec = {'Company': [],
                'Event_Date': [],
                'Avg_Next_Day_Returns': [],
                'Avg_Next_2_Days_Returns': [],
                'Avg_Next_5_Days_Returns': [],
                'Avg_Next_20_Days_Returns': []}
Next_Day_Returns=[]
Next_2_Days_Returns=[]
Next_5_Days_Returns=[]
Next_20_Days_Returns=[]


# Calculate average ARs and p-values for 10% threshold
for i in df_mid['Security-Symbol']:

    rows_with_condition = total_a_df[(total_a_df[f'{i}_Decrease_large_thres'] == 1) & (total_a_df['Market_Return_Decrease'] == 1)]

    for index, row in rows_with_condition.iterrows():

        # Date with large price change
        event_date = row['Date']

        # Index_next_day
        idx_next_day = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1

        # Index_next_2_days
        idx_next_2_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2
        next_2_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 2, f'{i}_ARs']
        list_2=next_2_days_returns.dropna().to_list()
        Next_2_Days_Returns.extend(list_2)

        # Index_next_5_days
        idx_next_5_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5
        next_5_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 5, f'{i}_ARs']
        list_5=next_5_days_returns.dropna().to_list()
        Next_5_Days_Returns.extend(list_5)

        # Index_next_20_days
        idx_next_20_days = total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20
        next_20_days_returns = total_a_df.loc[total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 1: total_a_df.index[total_a_df['Date'] == event_date].to_numpy()[0] + 20, f'{i}_ARs']
        list_20=next_20_days_returns.dropna().to_list()
        Next_20_Days_Returns.extend(list_20)


        if idx_next_day < len(total_a_df) and idx_next_2_days < len(total_a_df) and \
           idx_next_5_days < len(total_a_df) and idx_next_20_days < len(total_a_df):

            # Average_next_day
            next_day_returns = total_a_df.at[idx_next_day, f'{i}_ARs']
            Next_Day_Returns.append(next_day_returns)

            # Average_next_2_days
            avg_next_2_days_returns = next_2_days_returns.mean()

            # Average_next_5_days
            avg_next_5_days_returns = next_5_days_returns.mean()

            # Average_next_20_days
            avg_next_20_days_returns = next_20_days_returns.mean()


        results_dict_a_ten_decrease_alt_m_dec['Company'].append(i)
        results_dict_a_ten_decrease_alt_m_dec['Event_Date'].append(event_date)
        results_dict_a_ten_decrease_alt_m_dec['Avg_Next_Day_Returns'].append(avg_next_day_returns)
        results_dict_a_ten_decrease_alt_m_dec['Avg_Next_2_Days_Returns'].append(avg_next_2_days_returns)
        results_dict_a_ten_decrease_alt_m_dec['Avg_Next_5_Days_Returns'].append(avg_next_5_days_returns)
        results_dict_a_ten_decrease_alt_m_dec['Avg_Next_20_Days_Returns'].append(avg_next_20_days_returns)



        print(f'Company: {i}, Date: {event_date}, Avg Next Day Returns: {avg_next_day_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 2 Days Returns: {avg_next_2_days_returns} ')
        print(f'Company: {i}, Date: {event_date}, Avg Next 5 Days Returns: {avg_next_5_days_returns}')
        print(f'Company: {i}, Date: {event_date}, Avg Next 20 Days Returns: {avg_next_20_days_returns}')

results_significance_a_ten_decrease_alt_m_dec = pd.DataFrame(results_dict_a_ten_decrease_alt_m_dec)

results_significance_a_ten_decrease_alt_result_m_dec = {'Window':['1','1-2','1-5','1-20'],
                                                    'Avg_ARs_%':[],
                                                    'p_value':[]}
results_significance_a_ten_decrease_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_ten_decrease_alt_m_dec['Avg_Next_Day_Returns'].mean())
results_significance_a_ten_decrease_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_ten_decrease_alt_m_dec['Avg_Next_2_Days_Returns'].mean())
results_significance_a_ten_decrease_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_ten_decrease_alt_m_dec['Avg_Next_5_Days_Returns'].mean())
results_significance_a_ten_decrease_alt_result_m_dec['Avg_ARs_%'].append(results_significance_a_ten_decrease_alt_m_dec['Avg_Next_20_Days_Returns'].mean())

result_1=ttest_1samp(Next_Day_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_decrease_alt_result_m_dec['p_value'].append(result_1)

result_2=ttest_1samp(Next_2_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_decrease_alt_result_m_dec['p_value'].append(result_2)
result_5=ttest_1samp(Next_5_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_decrease_alt_result_m_dec['p_value'].append(result_5)
result_20=ttest_1samp(Next_20_Days_Returns,popmean=0,alternative='two-sided').pvalue
results_significance_a_ten_decrease_alt_result_m_dec['p_value'].append(result_20)
results_significance_a_ten_decrease_alt_result_m_dec=pd.DataFrame(results_significance_a_ten_decrease_alt_result_m_dec)

print(results_significance_a_eight_decrease_alt_result_m_dec.head())
print(results_significance_a_ten_decrease_alt_result_m_dec.head())


Company: ACAD.ST, Date: 2020-03-12 00:00:00, Avg Next Day Returns: -12.863918527645055
Company: ACAD.ST, Date: 2020-03-12 00:00:00, Avg Next 2 Days Returns: -5.04436210958691 
Company: ACAD.ST, Date: 2020-03-12 00:00:00, Avg Next 5 Days Returns: -0.44877507085113033
Company: ACAD.ST, Date: 2020-03-12 00:00:00, Avg Next 20 Days Returns: 1.352269727423933
Company: ALIG.ST, Date: 2020-03-16 00:00:00, Avg Next Day Returns: -12.863918527645055
Company: ALIG.ST, Date: 2020-03-16 00:00:00, Avg Next 2 Days Returns: -8.577213038153948 
Company: ALIG.ST, Date: 2020-03-16 00:00:00, Avg Next 5 Days Returns: -0.508047797257214
Company: ALIG.ST, Date: 2020-03-16 00:00:00, Avg Next 20 Days Returns: 1.43486972176535
Company: ALLIGO-B.ST, Date: 2020-03-12 00:00:00, Avg Next Day Returns: -12.863918527645055
Company: ALLIGO-B.ST, Date: 2020-03-12 00:00:00, Avg Next 2 Days Returns: -1.8534361510900244 
Company: ALLIGO-B.ST, Date: 2020-03-12 00:00:00, Avg Next 5 Days Returns: -8.265317659319875
Company: AL

In [22]:
results_significance_a_eight_increase_m.to_excel(f'{path}/raw_data/main/t3_results_significance_a_eight_increase_m_mid.xlsx')
results_significance_a_ten_increase_m.to_excel(f'{path}/raw_data/main/t3_results_significance_a_ten_increase_m_mid.xlsx')

results_significance_a_eight_increase_alt_result_m.to_excel(f'{path}/raw_data/main/t3_results_significance_a_eight_increase_alt_result_m_mid.xlsx')
results_significance_a_ten_increase_alt_result_m.to_excel(f'{path}/raw_data/main/t3_results_significance_a_ten_increase_alt_result_m_mid.xlsx')

results_significance_a_eight_increase_m_dec.to_excel(f'{path}/raw_data/main/t3_results_significance_a_eight_increase_m_dec_mid.xlsx')
results_significance_a_ten_increase_m_dec.to_excel(f'{path}/raw_data/main/t3_results_significance_a_ten_increase_m_dec_mid.xlsx')

results_significance_a_eight_increase_alt_result_m_dec.to_excel(f'{path}/raw_data/main/t3_results_significance_a_eight_increase_alt_result_m_mid_dec.xlsx')
results_significance_a_ten_increase_alt_result_m_dec.to_excel(f'{path}/raw_data/main/t3_results_significance_a_ten_increase_alt_result_m_mid_dec.xlsx')

results_significance_a_eight_decrease_m.to_excel(f'{path}/raw_data/main/t3_results_significance_a_eight_decrease_m_mid.xlsx')
results_significance_a_ten_decrease_m.to_excel(f'{path}/raw_data/main/t3_results_significance_a_ten_decrease_m_mid.xlsx')

results_significance_a_eight_decrease_alt_result_m.to_excel(f'{path}/raw_data/main/t3_results_significance_a_eight_decrease_alt_result_m_mid.xlsx')
results_significance_a_ten_decrease_alt_result_m.to_excel(f'{path}/raw_data/main/t3_results_significance_a_ten_decrease_alt_result_m_mid.xlsx')

results_significance_a_eight_decrease_m_dec.to_excel(f'{path}/raw_data/main/t3_results_significance_a_eight_decrease_m_dec_mid.xlsx')
results_significance_a_ten_decrease_m_dec.to_excel(f'{path}/raw_data/main/t3_results_significance_a_ten_decrease_m_dec_mid.xlsx')

results_significance_a_eight_decrease_alt_result_m_dec.to_excel(f'{path}/raw_data/main/t3_results_significance_a_eight_decrease_alt_result_m_dec_mid.xlsx')
results_significance_a_ten_decrease_alt_result_m_dec.to_excel(f'{path}/raw_data/main/t3_results_significance_a_ten_decrease_alt_result_m_dec_mid.xlsx')
